# Step 1: Evidence Details

|System|Source of data|Frameworks|Purpose|
|---|---|---|---|
|Okta|Okta Policies|Okta|Set administrative session timeout to 2 hours or less.|

```
RecomendedEvidenceName: AdministrativeSessionTimeoutReport
```



# Step 2: Define the System Specific Data


In [ ]:
# GET /api/v1/policies?type=OKTA_SIGN_ON
[
    {
        "id": "00pgromv1yFI7fVRP5d7",
        "status": "ACTIVE",
        "name": "Default Policy",
        "description": "The default policy applies in all situations if no other policy applies.",
        "priority": 1,
        "system": true,
        "conditions": {
            "people": {
                "groups": {
                    "include": [
                        "00ggromv1xSNSfOLy5d7"
                    ]
                }
            }
        },
        "created": "2024-04-30T03:17:35.000Z",
        "lastUpdated": "2024-04-30T03:17:35.000Z",
        "_links": {
            "self": {
                "href": "https://dev-93040161.okta.com/api/v1/policies/00pgromv1yFI7fVRP5d7",
                "hints": {
                    "allow": [
                        "GET",
                        "PUT"
                    ]
                }
            },
            "rules": {
                "href": "https://dev-93040161.okta.com/api/v1/policies/00pgromv1yFI7fVRP5d7/rules",
                "hints": {
                    "allow": [
                        "GET",
                        "POST"
                    ]
                }
            }
        },
        "type": "OKTA_SIGN_ON"
    }
]

# GET /api/v1/policies/{policyId}/rules
[
    {
        "id": "0prgromv1z68BXYQZ5d7",
        "status": "ACTIVE",
        "name": "Default Rule",
        "priority": 1,
        "created": null,
        "lastUpdated": "2024-04-30T03:17:35.000Z",
        "system": true,
        "conditions": {
            "people": {
                "users": {
                    "exclude": []
                }
            },
            "network": {
                "connection": "ANYWHERE"
            },
            "authContext": {
                "authType": "ANY"
            }
        },
        "actions": {
            "signon": {
                "access": "ALLOW",
                "requireFactor": false,
                "primaryFactor": "PASSWORD_IDP",
                "rememberDeviceByDefault": false,
                "session": {
                    "usePersistentCookie": false,
                    "maxSessionIdleMinutes": 120,
                    "maxSessionLifetimeMinutes": 0
                }
            }
        },
        "_links": {
            "self": {
                "href": "https://dev-93040161.okta.com/api/v1/policies/00pgromv1yFI7fVRP5d7/rules/0prgromv1z68BXYQZ5d7",
                "hints": {
                    "allow": [
                        "GET",
                        "PUT"
                    ]
                }
            }
        },
        "type": "SIGN_ON"
    }
]

# GET /api/v1/users
[
    {
        "id": "00ugromv83ZlFD4UC5d7",
        "status": "ACTIVE",
        "created": "2024-04-30T03:17:41.000Z",
        "activated": null,
        "statusChanged": "2024-04-30T05:11:26.000Z",
        "lastLogin": "2024-05-03T10:40:13.000Z",
        "lastUpdated": "2024-04-30T05:11:26.000Z",
        "passwordChanged": "2024-04-30T05:11:26.000Z",
        "type": {
            "id": "otygromv266cPwrj35d7"
        },
        "profile": {
            "firstName": "John",
            "lastName": "Doe",
            "mobilePhone": null,
            "secondEmail": null,
            "login": "john.doe@example.com",
            "email": "john.doe@example.com"
        },
        "credentials": {
            "password": {},
            "emails": [
                {
                    "value": "john.doe@example.com",
                    "status": "VERIFIED",
                    "type": "PRIMARY"
                }
            ],
            "provider": {
                "type": "OKTA",
                "name": "OKTA"
            }
        },
        "_links": {
            "self": {
                "href": "https://dev-93040161.okta.com/api/v1/users/00ugromv83ZlFD4UC5d7"
            }
        }
    }
]

# GET /api/v1/users/{userId}/roles
[
    {
        "id": "ra1gromv8evFERzm45d7",
        "label": "Super Administrator",
        "type": "SUPER_ADMIN",
        "status": "ACTIVE",
        "created": "2024-04-30T03:17:41.000Z",
        "lastUpdated": "2024-04-30T03:17:41.000Z",
        "assignmentType": "USER",
        "_links": {
            "assignee": {
                "href": "https://dev-93040161.okta.com/api/v1/users/00ugromv83ZlFD4UC5d7"
            }
        }
    }
]

# Step 2a: API & Flow


 - GET /api/v1/users
  - The response will give a list of users.
  - Filter out non-privileged users using below API.
 - GET /api/v1/users/{userId}/roles
  - The response will give a list of roles for the user.
  - If the list is empty, that means the user is not privileged.
  - If the list is not empty, that means the user has privileged access.
  - Thus, we got a list of privileged users.
 - GET /api/v1/policies?type=OKTA_SIGN_ON
  - The response will give the list of policies.
  - Loop through each policy to get policy details, & get policy ID which is required for the next API call.
 - GET /api/v1/policies/{policyId}/rules
  - The response will give the list of rules for the policy.
  - maxSessionLifetimeMinutes inside action must be less than the specified "MaxSessionLifetimeMinutes" value for atleast 1 rule on any policy for this rule to be compliant.
  - Create Extended Schema with the rule & policy details.
  - For each privileged user, check if atleast one compliant policy is attached to them.




# Step 2b: Define the Extended Schema


In [ ]:
# SignOnPolicies
# Input:
# {
#     "AllowedSessionLifetimeMinutes": 120
# }
[
    {
        "System": "okta",
        "Source": "compliancecow",
        "ResourceID": "00pgromv1yFI7fVRP5d7", # Policy ID
        "ResourceName": "Default Policy", # Policy name
        "ResourceType": "N/A",
        "ResourceLocation": "N/A",
        "ResourceTags": "N/A",
        "PolicyDescription": "The default policy applies in all situations if no other policy applies.",
        "PolicyStatus": "ACTIVE",
        "PolicyIncludedGroups": [
            "00ggromv2fMmkRAHZ5d7",
            "00ggromv1xSNSfOLy5d7"
        ],
        "PolicyExcludedGroups": [],
        "PolicyIncludedUsers": [],
        "PolicyExcludedUsers": [],
        "MaxSessionLifetimeMinutes": 120,
        "SigninFrequencyEnabled": True,
        "FrequencyIntervalIsEveryTime": False,
        "PolicyCreationDate": "2024-04-30T03:17:35.000Z",
    }
]

In [ ]:
# OktaAdminUsers
# Input:
# {
#     "AllowedSessionLifetimeMinutes": 120
# }
[
    {
        "System": "okta",
        "Source": "compliancecow",
        "ResourceID": "00ugromv83ZlFD4UC5d7",
        "ResourceName": "John Doe",
        "ResourceType": "N/A",
        "ResourceLocation": "N/A",
        "ResourceTags": "N/A",
        "ResourceURL": "https://<your-domain>.okta.com/admin/user/profile/view/00ugromv83ZlFD4UC5d7#tab-account"
        "UserEmail": "john.doe@example.com",
        "UserGroups":[{
            "GroupID": "00ugromv83ZlFD4UC5d9",
            "GroupName": "Default Group"
        }],
        "UserIsAdmin": True,
        "UserAdminRoles": [
            "Super Administrator"
        ],
        "UserCreatedDate": "2024-04-30T03:17:41.000Z",
    }
]

# Step 3: Define the Standard Schema
    

In [ ]:
# AdministrativeSessionTimeoutReport
# Input:
# {
#     "AllowedSessionLifetimeMinutes": 120
# }
[
    {
        "System": "okta",
        "Source": "compliancecow",
        "ResourceID": "00ugromv83ZlFD4UC5d7", # User ID
        "ResourceName": "John Doe", # User name
        "ResourceType": "N/A",
        "ResourceLocation": "N/A",
        "ResourceTags": "N/A",
        "UserEmail": "john.doe@example.com",
        "PolicyID": "00ugromv83ZlFD4UC5d7",
        "PolicyName": "Default Policy",
        "AllowedSessionLifetimeMinutes": 120,
        "MaxSessionLifetimeMinutes": 120,
        "SigninFrequencyEnabled": True,
        "FrequencyIntervalIsEveryTime": False,
        "ValidationStatusCode": "TIMEOUT_LT_OR_EQ_120_MIN",
        "ValidationStatusNotes": "The timeout for administrative sessions is set to less than 120 minutes",
        "ComplianceStatus": "COMPLIANT",
        "ComplianceStatusReason": "The timeout for administrative sessions is set to less than 120 minutes",
        "EvaluatedTime": "2024-04-23T07:54:43.012889Z",
        "UserAction": "",
        "ActionStatus": "",
        "ActionResponseURL": ""
    }
]

|System|Source|ResourceID|ResourceName|ResourceType|ResourceLocation|ResourceTags|UserEmail|PolicyID|PolicyName|AllowedSessionLifetimeMinutes|MaxSessionLifetimeMinutes|SigninFrequencyEnabled|FrequencyIntervalIsEveryTime|ValidationStatusCode|ValidationStatusNotes|ComplianceStatus|ComplianceStatusReason|EvaluatedTime|UserAction|ActionStatus|ActionResponseURL|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|okta|compliancecow|00ugromv83ZlFD4UC5d7|John Doe|N/A|N/A|N/A|john.doe@example.com|00ugromv83ZlFD4UC5d7|Default Policy|120|120|true|false|TIMEOUT_LT_120_MIN|The timeout for administrative sessions is set to less than 120 minutes|COMPLIANT|The timeout for administrative sessions is set to less than 120 minutes|2024-04-23T07:54:43.012889Z||||

# Step 4: Describe the Compliance Taxonomy

|Validation Status Code|Validation Status Notes|Compliance Status|Compliance Status Reason|
|---|---|---|--|
|TIMEOUT_LT_{n}_MIN| The timeout for administrative sessions is set to less than {n} minutes|COMPLIANT| The timeout for administrative sessions is set to less than {n} minutes. Your proactive management of session timeouts ensures enhanced security and efficiency.|
|POLICY_NA| The user does not have any compliant policy attached.|NON_COMPLIANT| The user does not have any compliant policy attached. It is recommended to attach a policy with a session timeout set to less than {allowed_session_lifetime_minutes} minutes for improved security and operational efficiency.|

# Step 5: Calculation for Compliance Percentage and Status




In [ ]:
# Calculation of Compliance Percentage
CompliancePCT = (100 - (Count of 'NON_COMPLIANT' records * 100) / Total records)

# Compliance Status
COMPLIANT - 100%
NON_COMPLIANT - 0%>=status<100%
NOT_DETERMINED - If some error occurs.

# Step 6: Describe (in words) the Remediation Steps for Non-Compliance

FOR NOT_DETERMINED : UNKNOWN

FOR COMPLIANT : None

For NONCOMPLIANCE:

If Compliance Cow needs to notify the client, the following message can be sent via slack or ticket raised in JIRA:

Please ensure that the administrative session timeout is set to 2 hours or less.

# 7. Control Setup Details

| Control Details            |                                                 |
|----------------------------|-------------------------------------------------|
| **RuleName**               | EnsureAdministrativeSessionTimeoutIsLTTwoHrs    |
| **PreRequisiteRuleNames**  | GetOktaUsers, GetOktaSignOnPolicies             |
| **ExtendedSchemaRuleNames**| GetOktaUsers, GetOktaSignOnPolicies             |
| **ApplicationClassName**   | oktaconnector                                   |
| **PostSynthesizerName**    | N/A                                             |